# Task
Set up and simulate a 3x3 grid world Markov Decision Process (MDP), including defining states, actions, rewards, and transition probabilities. Implement and evaluate a simple greedy policy for this MDP, and summarize the findings.

## Environment Setup

### Subtask:
Install the necessary Python libraries: `gymnasium` (or `gym`) for environment simulation and `numpy` for matrix operations. This ensures we have the tools required to build and interact with the MDP.


**Reasoning**:
The subtask requires installing `gymnasium` and `numpy` libraries. I will use `pip install` in a code block to install them.



In [1]:
pip install gymnasium numpy

## Define MDP Grid World

### Subtask:
Define the 3x3 grid world scenario. This includes defining the states (9 cells), actions (Up, Down, Left, Right), rewards (+10 for goal, -5 for danger, 0 otherwise), and the discount factor (0.9). We will also identify the goal and danger zones within the grid.


**Reasoning**:
I will define the grid dimensions, map states, define actions, create the reward structure, identify goal and danger states, and set the discount factor in a single Python code block, as outlined in the subtask instructions.



In [2]:
import numpy as np

# 1. Define grid dimensions
GRID_SIZE = (3, 3)
NUM_STATES = GRID_SIZE[0] * GRID_SIZE[1]

# 2. Map cells to state identifiers (0 to 8 for a 3x3 grid)
# We can represent states as a single integer from 0 to 8, or as (row, col) tuples.
# For simplicity, we'll use single integers and convert to (row, col) when needed.

def state_to_coords(state):
    return (state // GRID_SIZE[1], state % GRID_SIZE[1])

def coords_to_state(row, col):
    return row * GRID_SIZE[1] + col

print(f"Number of states: {NUM_STATES}")

# 3. Define the set of possible actions
ACTIONS = {'UP': 0, 'DOWN': 1, 'LEFT': 2, 'RIGHT': 3}
ACTIONS_NAMES = {0: 'UP', 1: 'DOWN', 2: 'LEFT', 3: 'RIGHT'}
NUM_ACTIONS = len(ACTIONS)

print(f"Available actions: {list(ACTIONS.keys())}")

# 4. Create a reward matrix or dictionary
# Let's define specific goal and danger zones first, then build the reward structure.

# 5. Identify the coordinates or state indices for the goal cell and any danger cells
GOAL_COORD = (0, 2)  # Top-right corner
DANGER_COORDS = [(1, 1)] # Center cell as danger

GOAL_STATE = coords_to_state(*GOAL_COORD)
DANGER_STATES = [coords_to_state(*coord) for coord in DANGER_COORDS]

# Initialize rewards for all states to 0
REWARDS = np.zeros(NUM_STATES)

# Set rewards for goal and danger states
REWARDS[GOAL_STATE] = 10
for d_state in DANGER_STATES:
    REWARDS[d_state] = -5

print(f"Goal state (coords: {GOAL_COORD}): {GOAL_STATE}, Reward: {REWARDS[GOAL_STATE]}")
print(f"Danger states (coords: {[state_to_coords(s) for s in DANGER_STATES]}): {DANGER_STATES}, Rewards: {REWARDS[DANGER_STATES]}")
print("Reward structure defined.")

# 6. Set the discount factor
DISCOUNT_FACTOR = 0.9

print(f"Discount factor (gamma): {DISCOUNT_FACTOR}")

Number of states: 9
Available actions: ['UP', 'DOWN', 'LEFT', 'RIGHT']
Goal state (coords: (0, 2)): 2, Reward: 10.0
Danger states (coords: [(1, 1)]): [4], Rewards: [-5.]
Reward structure defined.
Discount factor (gamma): 0.9


## Implement Transition Dynamics and Reward Function

### Subtask:
Write Python functions to model the MDP's core logic. One function will determine the next state given a current state and action, incorporating the 80% intended movement and 20% random movement transition probabilities. Another function will calculate the reward for transitioning to a specific state.


**Reasoning**:
The subtask requires implementing two Python functions: `get_next_state` to determine the probabilistic next states based on an action and `get_reward` to retrieve the reward for a given state. I will define these functions in a code block, incorporating boundary conditions and the specified transition probabilities for `get_next_state` and a direct lookup for `get_reward`.



In [3]:
import random

# Helper function to get valid next coordinates, handling grid boundaries
def _get_valid_next_coords(row, col, d_row, d_col):
    new_row, new_col = row + d_row, col + d_col
    if 0 <= new_row < GRID_SIZE[0] and 0 <= new_col < GRID_SIZE[1]:
        return new_row, new_col
    else:
        # If out of bounds, agent stays in the current cell
        return row, col

def get_next_state(state, action_idx):
    """
    Determines the probabilistic next states given a current state and an action.
    Transition probabilities: 80% intended, 10% for each perpendicular action.
    """
    current_row, current_col = state_to_coords(state)
    possible_transitions = []

    # Define movement offsets for actions
    move_offsets = {
        ACTIONS['UP']: (-1, 0),
        ACTIONS['DOWN']: (1, 0),
        ACTIONS['LEFT']: (0, -1),
        ACTIONS['RIGHT']: (0, 1)
    }

    # Define perpendicular actions for each main action
    perpendicular_actions = {
        ACTIONS['UP']: [ACTIONS['LEFT'], ACTIONS['RIGHT']],
        ACTIONS['DOWN']: [ACTIONS['LEFT'], ACTIONS['RIGHT']],
        ACTIONS['LEFT']: [ACTIONS['UP'], ACTIONS['DOWN']],
        ACTIONS['RIGHT']: [ACTIONS['UP'], ACTIONS['DOWN']]
    }

    # 1. Intended move (80% probability)
    dr_intended, dc_intended = move_offsets[action_idx]
    next_row_intended, next_col_intended = _get_valid_next_coords(current_row, current_col, dr_intended, dc_intended)
    next_state_intended = coords_to_state(next_row_intended, next_col_intended)
    possible_transitions.append((0.8, next_state_intended))

    # 2. Perpendicular moves (10% each)
    perp_action_1, perp_action_2 = perpendicular_actions[action_idx]

    # Perpendicular move 1
    dr_perp1, dc_perp1 = move_offsets[perp_action_1]
    next_row_perp1, next_col_perp1 = _get_valid_next_coords(current_row, current_col, dr_perp1, dc_perp1)
    next_state_perp1 = coords_to_state(next_row_perp1, next_col_perp1)
    possible_transitions.append((0.1, next_state_perp1))

    # Perpendicular move 2
    dr_perp2, dc_perp2 = move_offsets[perp_action_2]
    next_row_perp2, next_col_perp2 = _get_valid_next_coords(current_row, current_col, dr_perp2, dc_perp2)
    next_state_perp2 = coords_to_state(next_row_perp2, next_col_perp2)
    possible_transitions.append((0.1, next_state_perp2))

    # Combine probabilities for identical next states
    # This handles cases where different actions lead to the same 'next_state'
    # e.g., hitting a wall with an intended move, and a perpendicular move also hits the same wall or is invalid into the current state.
    merged_transitions = {}
    for prob, state_idx in possible_transitions:
        merged_transitions[state_idx] = merged_transitions.get(state_idx, 0.0) + prob

    return [(prob, state_idx) for state_idx, prob in merged_transitions.items()]

def get_reward(state):
    """
    Returns the reward associated with a specific state.
    """
    return REWARDS[state]

print("Functions `get_next_state` and `get_reward` defined.")

# Example Usage:
# Test from state 0 (top-left) with action RIGHT
# Expected: intended to state 1, perpendicular to 3 (DOWN) or 0 (UP, hit wall)
# current_state = 0 # (0,0)
# action = ACTIONS['RIGHT'] # 3
# transitions = get_next_state(current_state, action)
# print(f"Transitions from state {current_state} with action {ACTIONS_NAMES[action]}: {transitions}")

# Test reward for goal state
# print(f"Reward for GOAL_STATE ({GOAL_STATE}): {get_reward(GOAL_STATE)}")


Functions `get_next_state` and `get_reward` defined.


## Simulate Random Episodes

### Subtask:
Develop code to simulate several random episodes in the defined grid world. The robot will move through the environment based on random actions, and its path will be recorded for later review.


**Reasoning**:
The subtask requires simulating random episodes in the grid world, recording the path, and printing a summary for each episode. I will implement a Python code block that follows all the instructions given, including initializing `episode_history`, looping through episodes and steps, selecting random actions and next states based on probabilities, and storing the transitions.



In [4]:
import random

# 1. Initialize an empty list to store episode history
episode_history = []

# Parameters for simulation
NUM_EPISODES = 10
MAX_STEPS_PER_EPISODE = 20

print(f"Simulating {NUM_EPISODES} episodes...")

# 2. Loop to simulate a fixed number of episodes
for episode_num in range(NUM_EPISODES):
    episode_transitions = []

    # 3. Randomly choose a starting state, ensuring it's not a goal or danger state
    while True:
        start_state = random.randint(0, NUM_STATES - 1)
        if start_state != GOAL_STATE and start_state not in DANGER_STATES:
            break

    current_state = start_state
    done = False
    steps_taken = 0

    # 5. Inner loop to simulate steps within an episode
    while not done and steps_taken < MAX_STEPS_PER_EPISODE:
        # 6a. Select a random action
        action_idx = random.choice(list(ACTIONS.values()))
        action_name = ACTIONS_NAMES[action_idx]

        # 6b. Use get_next_state to get possible next states and their probabilities
        possible_transitions = get_next_state(current_state, action_idx)

        # 6c. Randomly select one next_state based on probabilities
        # Unzip the list of (probability, state) tuples into two separate lists
        probabilities, states = zip(*possible_transitions)
        # Use random.choices to select a state based on the provided probabilities
        next_state = random.choices(states, weights=probabilities, k=1)[0]

        # 6d. Get the reward for the next_state
        reward = get_reward(next_state)

        # 6e. Append the tuple to episode_transitions
        episode_transitions.append((current_state, action_name, reward, next_state))

        # 6f. Update current_state to next_state
        current_state = next_state

        # 6g. Check if the current_state is a terminal state
        if current_state == GOAL_STATE or current_state in DANGER_STATES:
            done = True

        steps_taken += 1

    episode_history.extend(episode_transitions) # Add current episode's transitions to overall history

    # 7. Print a summary of the episode
    print(f"Episode {episode_num + 1} finished. Total steps: {steps_taken}. Final state: {current_state} ({'Goal' if current_state == GOAL_STATE else 'Danger' if current_state in DANGER_STATES else 'In progress'}) from start state: {start_state}")

print("Simulation complete. Episode history stored.")

Simulating 10 episodes...
Episode 1 finished. Total steps: 12. Final state: 2 (Goal) from start state: 8
Episode 2 finished. Total steps: 4. Final state: 4 (Danger) from start state: 1
Episode 3 finished. Total steps: 1. Final state: 4 (Danger) from start state: 7
Episode 4 finished. Total steps: 8. Final state: 2 (Goal) from start state: 1
Episode 5 finished. Total steps: 20. Final state: 6 (In progress) from start state: 0
Episode 6 finished. Total steps: 20. Final state: 6 (In progress) from start state: 5
Episode 7 finished. Total steps: 20. Final state: 2 (Goal) from start state: 3
Episode 8 finished. Total steps: 14. Final state: 4 (Danger) from start state: 3
Episode 9 finished. Total steps: 1. Final state: 4 (Danger) from start state: 5
Episode 10 finished. Total steps: 12. Final state: 4 (Danger) from start state: 7
Simulation complete. Episode history stored.


## Visualize Robot Movement (Optional)

### Subtask:
Create a simple visualization (e.g., using print statements or a text-based grid representation) to show the robot's movement through the grid during a simulated episode. This will help in understanding the environment's dynamics.


## Visualize Robot Movement (Optional)

### Subtask:
Create a simple visualization (e.g., using print statements or a text-based grid representation) to show the robot's movement through the grid during a simulated episode. This will help in understanding the environment's dynamics.

#### Instructions
1. Define a function, say `print_grid_state(current_state, action_name, next_state)`, that takes the current state, the action taken, and the next state as input.
2. Inside this function, create a 3x3 grid representation (e.g., a list of lists or a 2D array).
3. Mark the goal state ('G'), danger states ('D'), and the robot's current position ('R') on the grid. If the current position is also a goal or danger state, you can prioritize 'R' or indicate both (e.g., 'RG' or 'RD').
4. Print the grid, along with information about the current state, action, and next state.
5. Select one or more episodes from the `episode_history` (e.g., the first episode or an episode that reached the goal/danger state) for visualization.
6. Iterate through the transitions of the selected episode(s) and call the `print_grid_state` function for each transition to visualize the robot's movement step by step.

**Reasoning**:
The subtask requires implementing a function `print_grid_state` to visualize the robot's movement in the grid and then iterating through selected episodes from `episode_history` to display the steps. This will be done in a code block.



In [5]:
def print_grid_state(current_state, action_name, next_state):
    """
    Prints a 3x3 grid representation of the MDP, marking goal, danger, and robot's position.
    """
    grid = [['.' for _ in range(GRID_SIZE[1])] for _ in range(GRID_SIZE[0])]

    # Mark Goal state
    g_row, g_col = state_to_coords(GOAL_STATE)
    grid[g_row][g_col] = 'G'

    # Mark Danger states
    for d_s in DANGER_STATES:
        d_row, d_col = state_to_coords(d_s)
        if grid[d_row][d_col] == '.': # Avoid overwriting 'G' if by chance goal is also danger
            grid[d_row][d_col] = 'D'

    # Mark Robot's current position
    r_row, r_col = state_to_coords(current_state)
    if grid[r_row][r_col] == 'G':
        grid[r_row][r_col] = 'RG' # Robot is on Goal
    elif grid[r_row][r_col] == 'D':
        grid[r_row][r_col] = 'RD' # Robot is on Danger
    else:
        grid[r_row][r_col] = 'R'

    print(f"\n--- Step ---")
    print(f"Current State: {current_state} (R)")
    print(f"Action Taken: {action_name}")
    print(f"Next State: {next_state}")
    print("Grid:")
    for row in grid:
        print(' '.join(row))
    print("------------")

print("Function `print_grid_state` defined.")

# Visualize a specific episode (e.g., the first one)
if episode_history:
    print(f"\nVisualizing the first episode from `episode_history` (Total steps: {len(episode_history)})...")
    # For simplicity, let's visualize the first 'MAX_STEPS_PER_EPISODE' steps or until a terminal state
    # if the episode history contains more than one episode's steps, we need to filter.
    # Assuming episode_history stores all transitions sequentially.
    # Let's take the first complete episode. An episode ends when current_state is a terminal state
    # or MAX_STEPS_PER_EPISODE is reached.

    # Find the end of the first episode
    first_episode_transitions = []
    for i, transition in enumerate(episode_history):
        first_episode_transitions.append(transition)
        # A transition is (current_state, action_name, reward, next_state)
        # The next_state is the state reached after the action.
        # Check if the next_state is terminal
        if transition[3] == GOAL_STATE or transition[3] in DANGER_STATES:
            break

    for current_s, action_n, reward_val, next_s in first_episode_transitions:
        print_grid_state(current_s, action_n, next_s)
    print(f"Visualization of first episode completed.")
else:
    print("No episode history to visualize.")


Function `print_grid_state` defined.

Visualizing the first episode from `episode_history` (Total steps: 112)...

--- Step ---
Current State: 8 (R)
Action Taken: RIGHT
Next State: 5
Grid:
. . G
. D .
. . R
------------

--- Step ---
Current State: 5 (R)
Action Taken: UP
Next State: 5
Grid:
. . G
. D R
. . .
------------

--- Step ---
Current State: 5 (R)
Action Taken: DOWN
Next State: 8
Grid:
. . G
. D R
. . .
------------

--- Step ---
Current State: 8 (R)
Action Taken: LEFT
Next State: 7
Grid:
. . G
. D .
. . R
------------

--- Step ---
Current State: 7 (R)
Action Taken: LEFT
Next State: 6
Grid:
. . G
. D .
. R .
------------

--- Step ---
Current State: 6 (R)
Action Taken: LEFT
Next State: 6
Grid:
. . G
. D .
R . .
------------

--- Step ---
Current State: 6 (R)
Action Taken: RIGHT
Next State: 7
Grid:
. . G
. D .
R . .
------------

--- Step ---
Current State: 7 (R)
Action Taken: RIGHT
Next State: 8
Grid:
. . G
. D .
. R .
------------

--- Step ---
Current State: 8 (R)
Action Take

## Implement Simple Greedy Policy

### Subtask:
Create a basic greedy policy that, from any given state, chooses the action that leads to the highest immediate reward. This will be a first step towards solving the MDP.


**Reasoning**:
I will define the `greedy_policy(state)` function as per the subtask instructions. This function will iterate through all possible actions, use `get_next_state` to find the most probable next state for each action, and then `get_reward` to determine the immediate reward. Finally, it will return the action that leads to the maximum immediate reward. I will also include a print statement to indicate that the function has been defined.

**Reasoning**:
I will implement the `greedy_policy` function as described in the subtask instructions, which involves iterating through actions, finding the most probable next state, calculating its reward, and selecting the action that maximizes this immediate reward. I will also add a print statement to confirm the function definition.



In [6]:
def greedy_policy(state):
    """
    A simple greedy policy that chooses the action leading to the highest immediate reward.
    It considers only the most probable next state (80% intended move).
    """
    max_immediate_reward = -float('inf')
    best_action_idx = None

    for action_idx in ACTIONS.values():
        # Get possible next states and their probabilities for the current action
        possible_transitions = get_next_state(state, action_idx)

        # Find the most probable next state (assuming 80% intended move)
        # The `get_next_state` function returns tuples of (probability, state_idx).
        # We need to find the state with the highest probability.
        most_probable_next_state = None
        highest_prob = -1.0

        for prob, next_s in possible_transitions:
            if prob > highest_prob:
                highest_prob = prob
                most_probable_next_state = next_s

        # If for some reason no state was found (shouldn't happen with our current logic),
        # or if the current state is terminal, handle appropriately. For non-terminal states,
        # a most probable next state will always exist.
        if most_probable_next_state is not None:
            # Get the reward for this most probable next state
            current_reward = get_reward(most_probable_next_state)

            # Update best action if this one yields a higher immediate reward
            if current_reward > max_immediate_reward:
                max_immediate_reward = current_reward
                best_action_idx = action_idx

    # If all actions lead to equally low or zero reward, or no next state could be determined,
    # choose a random action as a fallback, or the first one.
    if best_action_idx is None: # This could happen if all transitions lead to terminal states with no reward, or if the state itself is terminal.
        best_action_idx = random.choice(list(ACTIONS.values()))

    return best_action_idx

print("Function `greedy_policy` defined.")

# Example Usage:
# Test the greedy policy from an arbitrary state, e.g., state 0
# test_state = 0
# recommended_action_idx = greedy_policy(test_state)
# print(f"Greedy policy from state {test_state} recommends action: {ACTIONS_NAMES[recommended_action_idx]}")

# Test from a state adjacent to goal to see if it moves towards goal
# test_state_near_goal = 5 # (1,2)
# recommended_action_idx_near_goal = greedy_policy(test_state_near_goal)
# print(f"Greedy policy from state {test_state_near_goal} (near goal) recommends action: {ACTIONS_NAMES[recommended_action_idx_near_goal]}")


Function `greedy_policy` defined.


## Evaluate Greedy Policy

### Subtask:
Evaluate the performance of the implemented greedy policy. Simulate several episodes where the robot follows this policy, record its path, total reward, and whether it reached a terminal state (goal or danger). Summarize the findings to understand the policy's effectiveness.

## Evaluate Greedy Policy

### Subtask:
Evaluate the performance of the implemented greedy policy. Simulate several episodes where the robot follows this policy, record its path, total reward, and whether it reached a terminal state (goal or danger). Summarize the findings to understand the policy's effectiveness.

## Evaluate Greedy Policy

### Subtask:
Evaluate the performance of the implemented greedy policy. Simulate several episodes where the robot follows this policy, record its path, total reward, and whether it reached a terminal state (goal or danger). Summarize the findings to understand the policy's effectiveness.

**Reasoning**:
The subtask requires simulating episodes using the greedy policy, recording the path, total reward, and terminal state for each episode. I will implement a Python code block that iterates through a specified number of episodes and steps, using the `greedy_policy` function to choose actions and then `get_next_state` and `get_reward` to update the environment. A summary of each episode will be printed.



In [7]:
greedy_episode_history = []
NUM_GREEDY_EPISODES = 10
MAX_GREEDY_STEPS = 50 # Allow more steps to see if it reaches goal/danger

print(f"Simulating {NUM_GREEDY_EPISODES} episodes using the greedy policy...")

for episode_num in range(NUM_GREEDY_EPISODES):
    episode_transitions = []
    total_reward = 0

    # Randomly choose a starting state, ensuring it's not a goal or danger state
    while True:
        start_state = random.randint(0, NUM_STATES - 1)
        if start_state != GOAL_STATE and start_state not in DANGER_STATES:
            break

    current_state = start_state
    done = False
    steps_taken = 0

    while not done and steps_taken < MAX_GREEDY_STEPS:
        # Choose action based on the greedy policy
        action_idx = greedy_policy(current_state)
        action_name = ACTIONS_NAMES[action_idx]

        # Get possible next states and their probabilities
        possible_transitions = get_next_state(current_state, action_idx)

        # Randomly select one next_state based on probabilities
        probabilities, states = zip(*possible_transitions)
        next_state = random.choices(states, weights=probabilities, k=1)[0]

        # Get the reward for the next_state
        reward = get_reward(next_state)
        total_reward += reward

        # Append the transition to history
        episode_transitions.append((current_state, action_name, reward, next_state))

        # Update current_state
        current_state = next_state

        # Check for terminal states
        if current_state == GOAL_STATE or current_state in DANGER_STATES:
            done = True

        steps_taken += 1

    greedy_episode_history.extend(episode_transitions)

    final_state_type = 'In progress'
    if current_state == GOAL_STATE:
        final_state_type = 'Goal'
    elif current_state in DANGER_STATES:
        final_state_type = 'Danger'

    print(f"Episode {episode_num + 1} (Greedy) finished. Steps: {steps_taken}. Total Reward: {total_reward:.2f}. Final state: {current_state} ({final_state_type}) from start state: {start_state}")

print("Greedy policy simulation complete. History stored in `greedy_episode_history`.")

Simulating 10 episodes using the greedy policy...
Episode 1 (Greedy) finished. Steps: 2. Total Reward: 10.00. Final state: 2 (Goal) from start state: 8
Episode 2 (Greedy) finished. Steps: 1. Total Reward: 10.00. Final state: 2 (Goal) from start state: 1
Episode 3 (Greedy) finished. Steps: 1. Total Reward: 10.00. Final state: 2 (Goal) from start state: 5
Episode 4 (Greedy) finished. Steps: 2. Total Reward: 10.00. Final state: 2 (Goal) from start state: 8
Episode 5 (Greedy) finished. Steps: 1. Total Reward: 10.00. Final state: 2 (Goal) from start state: 5
Episode 6 (Greedy) finished. Steps: 12. Total Reward: 10.00. Final state: 2 (Goal) from start state: 7
Episode 7 (Greedy) finished. Steps: 5. Total Reward: 10.00. Final state: 2 (Goal) from start state: 7
Episode 8 (Greedy) finished. Steps: 3. Total Reward: 10.00. Final state: 2 (Goal) from start state: 8
Episode 9 (Greedy) finished. Steps: 1. Total Reward: 10.00. Final state: 2 (Goal) from start state: 5
Episode 10 (Greedy) finished. S

## Final Task

### Subtask:
Provide a summary of the implemented MDP, the simulated episodes, and the performance of the simple greedy policy, discussing key findings and potential next steps for policy improvement.


## Summary:

### Q&A
*   **What is the summary of the implemented MDP?**
    The implemented MDP is a 3x3 grid world with 9 states. Actions include 'UP', 'DOWN', 'LEFT', and 'RIGHT'. The goal state is (0,2) (state 2) with a reward of +10, and a danger state is (1,1) (state 4) with a reward of -5. All other states have a reward of 0. The discount factor is 0.9. Transition dynamics are probabilistic: 80% chance of intended movement and 10% chance for each of the two perpendicular movements, with boundary conditions causing the agent to stay in the current cell if it moves out of bounds.

*   **What is the summary of the simulated episodes?**
    Initially, 10 random episodes were simulated, with a maximum of 20 steps per episode. These episodes demonstrated varied outcomes: some reached the goal, some hit danger states, and others terminated by reaching the maximum step limit without hitting a terminal state. Later, 10 episodes were simulated using a simple greedy policy, with a maximum of 50 steps per episode.

*   **What is the performance of the simple greedy policy?**
    The simple greedy policy performed reasonably well. Out of 10 simulated episodes, 9 successfully reached the Goal state (state 2) with a total reward of 10.00 each. One episode ended in a Danger state (state 4) with a total reward of -5.00. The number of steps taken to reach the goal varied from 1 to 12 steps.

*   **What are the key findings?**
    The key findings indicate that the simple greedy policy, which prioritizes immediate highest reward, is generally effective in guiding the agent towards the goal state in this probabilistic environment, despite occasionally leading to danger due to the random components of state transitions.

*   **What are potential next steps for policy improvement?**
    Potential next steps for policy improvement include implementing more sophisticated policy learning algorithms such as Value Iteration or Q-learning, which consider long-term discounted rewards rather than just immediate rewards, and potentially incorporating an exploration-exploitation strategy.

### Data Analysis Key Findings
*   The MDP consists of a 3x3 grid (9 states), 4 actions (UP, DOWN, LEFT, RIGHT), a goal state (state 2) with a reward of +10, and a danger state (state 4) with a reward of -5. The discount factor is 0.9.
*   Transition probabilities involve an 80% chance for the intended action and 10% for each of the two perpendicular actions, with boundary conditions handling out-of-bounds movements by keeping the agent in its current state.
*   Initial random simulations showed that agents can reach goal states, danger states, or exhaust the maximum step limit (20 steps) without reaching a terminal state.
*   The implemented simple greedy policy, which selects actions based on the immediate reward of the most probable next state, was evaluated over 10 episodes.
*   The greedy policy led to reaching the Goal state (state 2) in 9 out of 10 episodes, achieving a total reward of 10.00 for each successful episode.
*   One episode under the greedy policy resulted in reaching the Danger state (state 4), yielding a total reward of -5.00.
*   The number of steps taken to reach a terminal state by the greedy policy varied, ranging from 1 to 12 steps for successful episodes.

### Insights or Next Steps
*   The simple greedy policy, while effective in many cases, is not optimal due to the probabilistic transitions and its sole focus on immediate rewards. It fails to account for potential future negative rewards or longer paths that might lead to a larger cumulative discounted reward.
*   To improve policy performance and robustness, implement and evaluate more advanced reinforcement learning algorithms such as Value Iteration or Q-learning. These methods consider the long-term expected discounted reward, which is crucial in stochastic environments, potentially leading to a policy that avoids danger states more reliably and finds more efficient paths to the goal.
